# Base du fonctionnement

Le debut de ce notebook provient majoritairement de :
 - [La documentation officielle](https://www.tensorflow.org/) 
 - [Xebia ep1](http://blog.xebia.fr/2017/03/01/tensorflow-deep-learning-episode-1-introduction/)


Notebook créer à des fins d'utilisation personnelle 

In [1]:
from IPython.display import Image
from IPython.core.display import HTML
import numpy as np
import tensorflow as tf

## TensorFlow, mais qu’est-ce donc ?

TensorFlow est un framework de programmation pour le calcul numérique qui a été rendu Open Source par Google en Novembre 2015. Depuis sa release, TensorFlow n’a cessé de gagner en popularité, pour devenir très rapidement l’un des frameworks les plus utilisés pour le Deep Learning, comme le montrent les dernières comparaisons suivantes, faites par François Chollet (auteur de la librairie Keras).

In [3]:
Image(url= "http://blog.xebia.fr/wp-content/uploads/2017/03/deeplearning-1-768x413.jpg")

Les raisons de cette popularité fracassante ?
- Multi-plateformes (Linux, Mac OS, et même Android et iOS !)
- APIs en Python, C++, Java et Go (l’API Python est plus complète cependant, c’est sur celle-ci que nous allons travailler)
- Temps de compilation très courts dû au backend en C/C++
- Supporte les calculs sur CPU, GPU et même le calcul distribué sur cluster
- Une documentation extrêmement bien fournie avec de nombreux exemples et tutoriels
- Last but not least: Le fait que le framework vienne de Google et que ce dernier ait annoncé avoir migré la quasi totalité de ses projets liés au Deep Learning en TensorFlow est quelque peu rassurant

## Comment ça marche ?

La particularité de TensorFlow est qu’il représente les calculs sous la forme d’un graphe d’exécution: chaque noeud représente une Operation à réaliser, et chaque lien représente un Tensor. Une Operation peut aller d’une simple addition à une fonction complexe de différenciation matricielle.

Chaque Operation prend en entrée zéro, un ou plusieurs Tensor, effectue un calcul, et retourne zéro, un ou plusieurs Tensor. Un exemple typique de Tensor est un batch d’images. Un batch d’images est représenté par un Tensor à 4 dimensions: taille du batch (nombre d’images dans le batch), hauteur, largeur et nombre de canaux de représentation (3 pour une image en couleurs représentée en RGB).

La création du graphe est automatiquement gérée par TensorFlow une fois les Tensor et Operation implémentés et instanciés. Cela permet une optimisation et parallélisation du code et de l’exécution lors du lancement.

TensorFlow possède de plus un support très vaste pour la création d’opérations spécifiques au Deep Learning, et il devient donc facile de construire un réseau de neurones et d’utiliser les opérations mathématiques couramment associées pour l’entraîner avec les bons optimiseurs.

## Comment les calculs sont-ils effectués ?

### La Session

Pour pouvoir exécuter quoi que ce soit, un graphe doit être lancé dans une Session. Une Session place les Operation du graphe dans des devices (CPU ou GPU) et met à disposition des méthodes pour les exécuter. Chaque Session peut avoir ses propres variables et readers, et il est possible d’instancier plusieurs Session afin d’entraîner plusieurs réseaux différents. Le lancement des opérations du graphe se fait via la méthode run() de la Session. Un graphe ne va exécuter les Operation qu’après la création d’une Session.

Ce système d’exécution de graphe est une des propriétés fondamentales de TensorFlow. Cela permet d’éviter de retourner dans le monde Python à chaque étape (contrairement à ce qui est fait dans NumPy) et d’éxécuter toutes les opérations du graphe en une seule fois dans un même backend optimisé.

L’utilisation d’une Session n’est pas ce qu’il y a de plus intuitif et simple, en particulier lorsque l’on utilise des notebooks et que l’on ne souhaite pas lancer le run de la Session à chaque fois que l’on veut tester une étape. Pour s’affranchir de ces contraintes lors d’une utilisation via des notebooks, on peut utiliser à la place une InteractiveSession, qui remplit les mêmes fonctions.

### Structure du code

Généralement, le code associé à TensorFlow se divise donc en deux étapes principales:
 - Une phase de construction durant laquelle on décrit et assemble toutes les variables et opérations du graphe
 - Une phase d’exécution qui utilise une Session afin d’exécuter les opérations du graphe
 
C’est cette séparation qui permet à TensorFlow d’optimiser l’enchaînement des étapes du graphe avant de les exécuter.


## Les opérations courantes pour gérer les inputs

### Constant
**tf.constant()** pour créer un Tensor à partir d’une valeur que l’on souhaite garder fixe.

In [8]:
# Instanciation of a Constant
const = tf.constant([2., 1.])

In [7]:
# Create a Constant op that produces a 1x2 matrix.
matrix1 = tf.constant([[3., 3.]])

# Create another Constant that produces a 2x1 matrix.
matrix2 = tf.constant([[2.], [2.]])

# Create a matmul op that performs the matrix multiplication of matrix1 by matrix2.
product = tf.matmul(matrix1, matrix2)

# Launch the default graph.
sess = tf.Session()

# Call the session run() method to run the matmul op.
result = sess.run(product)
print(result)

# You can call multiple operations at the same time
res_product, res_matrix1 = sess.run([product, matrix1])
print(res_matrix1)

# Close the session
sess.close()

[[ 12.]]
[[ 3.  3.]]


### Variable
**tf.Variable()** pour créer un Tensor que l'on souhaite modifier au cours de l'éxécution d'un graphe

**tf.global_variables_initializer()** pour initializer les variables avant de les utiliser dans un graphe

In [9]:
# Instanciation of a Variable
counter = tf.Variable(0, name="counter")  # Count the number of iterations

# Initialization of a Variable as a Tensor full of zeros
weights = tf.Variable(tf.zeros([image_pixels, num_classes]))

# Initialization of a Variable as a Tensor with small random values
weights = tf.Variable(tf.truncated_normal(shape=[num_pixels, num_classes], stddev=0.1))

In [11]:
# Counter Variable definition
counter = tf.Variable(0, name="counter")

# Creation of a constant
one = tf.constant(1)

# Operations to perform in order to increment the variable value
new_value = tf.add(counter, one)
update = tf.assign(counter, new_value)

# Initialize all variables
init_op = tf.global_variables_initializer()

# Increment the value of the variable in a session
with tf.Session() as sess:
    sess.run(init_op)
    for _ in range(5):
        sess.run(update)
        print(sess.run(counter))

1
2
3
4
5


### Placeholder
**tf.placeholder()** pour créer un tensor sans valeur spécifique, sa valeur sera fixé lors d'un run grâce à l'argument **feed_dict**. Sert principalement pour les batchs de données successives.

In [ ]:
# Instanciation of a Placeholder
simple_placeholder = tf.placeholder(tf.float32)

In [10]:
# Instanciation of two Placeholders
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)

# Multiplication operation
output = tf.multiply(input1, input2)

# Graph execution, we need to feed the placeholders
with tf.Session() as sess:
 result = sess.run(output, feed_dict={input1: [7.], input2: [2.]})
 print(result)

[ 14.]


### Basics operations

In [3]:
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [4]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [5]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

In [8]:
print(sess.run(adder_node, {a: 3, b:4.5}))
print(sess.run(adder_node, {a: [1,3], b: [2, 4]}))
print(adder_node.eval({a: [1,3], b: [2, 4]},sess))

7.5
[ 3.  7.]
[ 3.  7.]


Showing somes basics operation and performing a simple regression with standard operations with TensorFlow Core

In [40]:
# Classic way of seperation of the code into graph declaration and session

# Input data
x_train = [1,3,5,7]
y_train = [1,2,3,12]

graph = tf.Graph()
with graph.as_default():
    
    # Model parameters
    W = tf.Variable([.3], tf.float32)
    b = tf.Variable([-.3], tf.float32)
    
    # Model input and output
    x = tf.placeholder(tf.float32)
    y = tf.placeholder(tf.float32)
    
    # Operation
    linear_model = W * x + b
    squared_deltas = tf.square(linear_model - y)
    
    # Loss
    sum_loss = tf.reduce_sum(squared_deltas) # Take the sum
    mean_loss = tf.reduce_mean(squared_deltas) # Take the mean
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = optimizer.minimize(mean_loss)
    
with tf.Session(graph=graph) as session:
    
    # Initialized variables
    init = tf.global_variables_initializer() # before no values is assign to them
    session.run(init)
    #tf.global_variables_initializer().run()
    
    # Using eval
    print('linear_model:',linear_model.eval({x: x_train}))
    print('squared_deltas:',squared_deltas.eval({x: x_train, y: y_train}))
    print('sum_loss:',sum_loss.eval({x: x_train, y:y_train}))
    print('mean_loss:',mean_loss.eval({x: x_train, y:y_train}))
    print()
    
    # Re-assign value to W and b
    fixW = tf.assign(W, [-1.]) 
    fixb = tf.assign(b, [1.])
    session.run([fixW,fixb]) # Even if we give an another name the value will be assign to the ref variable after the run
    
    # Can do the same thing with one call of session.run()
    linear_mod, squared_del, sum_los, mean_los = session.run([linear_model, squared_deltas, sum_loss, mean_loss], 
                                               feed_dict={x: x_train, y:y_train})
    print('linear_model:', linear_mod)
    print('squared_deltas:', squared_del)
    print('sum_loss:', sum_los)
    print('mean_loss:', mean_los)
    print()
    
    # Training loop
    for i in range(1000):
        session.run(train, {x:x_train, y:y_train})
    
    # Evaluation of the results
    linear_mod, squared_del, sum_los, mean_los = session.run([linear_model, squared_deltas, sum_loss, mean_loss], 
                                               feed_dict={x: x_train, y:y_train})
    print('linear_model:', linear_mod)
    print('squared_deltas:', squared_del)
    print('sum_loss:', sum_los)
    print('mean_loss:', mean_los)
    print()
    

linear_model: [ 0.          0.60000002  1.20000005  1.80000019]
squared_deltas: [   1.            1.95999992    3.23999977  104.03999329]
sum_loss: 110.24
mean_loss: 27.56

linear_model: [ 0. -2. -4. -6.]
squared_deltas: [   1.   16.   49.  324.]
sum_loss: 390.0
mean_loss: 97.5

linear_model: [-0.57022488  2.81557131  6.20136786  9.58716393]
squared_deltas: [  2.46560621   0.66515654  10.24875641   5.82177782]
sum_loss: 19.2013
mean_loss: 4.80032



Simple regression with standard operations with the high-level TensorFlow library tf.contrib.learn

In [41]:
# Data input and output
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])

# Declare list of features.
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]

# Declare the estimator
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

# Helper method to read and set up data sets, and tell the function how many batches
# of data (num_epochs) we want and how big each batch should be
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x}, y, batch_size=4,
                                              num_epochs=1000)

estimator.fit(input_fn=input_fn, steps=1000)

estimator.evaluate(input_fn=input_fn)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_tf_random_seed': None, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_task_id': 0, '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000001D9A3F60>, '_master': ''}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\MASTER~

{'global_step': 1000, 'loss': 1.838383e-09}

## TensorBoard

Nous allons très vite ressentir le besoin de visualiser le graphe créé, ainsi que de contrôler l’évolution de nos phases d’apprentissage (évolution du taux de prédiction, activités des neurones, etc.). Heureusement pour nous, TensorFlow met à disposition un outil, TensorBoard, qui répond à ces besoins. TensorBoard est une réelle force et constitue un vrai élément différenciant de TensorFlow par rapport aux autres frameworks de Deep Learning.

Lien: 
- https://github.com/tensorflow/tensorflow/blob/master/tensorflow/tensorboard/README.md
- https://www.tensorflow.org/get_started/summaries_and_tensorboard
- https://www.tensorflow.org/get_started/graph_viz
- https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/tutorials/deepdream
- https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/deepdream/deepdream.ipynb
- https://medium.com/intuitionmachine/teasing-out-tensorflow-graph-mess-64cf5ece4b00

### Comment utiliser TensorBoard ?

Pour utiliser TensorBoard, il faut spécifier dans le code quelles sont les opérations dont on souhaite résumer l’activité. Une fois que c’est fait, il reste alors à créer un Summarizer qui va merger toutes les informations calculées. Une fois que le programme est lancé, il suffit de lancer la commande adéquate avec le chemin d’accès vers les logs pour que l’interface graphique associée se lance.

In [16]:
# During the session part
file_writer = tf.summary.FileWriter('path/to/logs', sess.graph)

# In a terminal
tensorboard --logdir='path/to/logs'

### From deep dream notebook, using Tensorboard insisde Ipython Notebook

In [24]:
# Helper function from Alex Mordvintsev deep dream notebook
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
#Then to visualize current graph
show_graph(tf.get_default_graph().as_graph_def())

# If your graph is saved as pbtxt, you could do
gdef = tf.GraphDef()
from google.protobuf import text_format
text_format.Merge(open("tf_persistent.pbtxt").read(), gdef)
show_graph(gdef)

### Les modifications à apporter

Afin de pouvoir visualiser l’évolution de certains nœuds du graphe, il faut annoter l’opération correspondante via des Summary Operations. On trouve par exemple **tf.summary.scalar** (ex: visualisation d’un fonction de coût) ou des **tf.summary.histogram** (ex: visualisation de la distribution de l’activation des neurones d’une couche).

Comme on peut le voir dans l’exemple, le résumé des valeurs prises par la variable counter est géré par l’opération **tf.summary.scalar(‘counter’, counter)**. La création de la Variable ainsi que le résumé associé sont encapsulés dans un bloc with **tf.name_scope**. Cela permet de générer des grandes zones dans TensorBoard, ce qui nous sera très utile lorsque nous aurons beaucoup d’opérations enchainées.

Une fois les summarizers créés, on ajoute une dernière opération chargée de tous les merger: **tf.summary.merge_all()**. 

L’étape suivante consiste à créer un summary_writer spécifiant où seront écrits les logs qui seront lus par TensorBoard: **tf.summary.FileWriter()**. Lors de l’éxécution du graphe, il suffit alors de rajouter à l’étape de run l’opération de merge, puis d’ajouter le summary au writer créé: **summary_writer.add_summary(summary, i)**.

Ces quelques lignes de code supplémentaires nous permettent de faire appel à TensorBoard en ligne de commande pour lancer l’interface graphique associée. On observe alors plusieurs onglets, notamment « Scalars » et « Graph ».

In [28]:
# creating TensorFlow session and loading the model
graph = tf.Graph()
#sess = tf.InteractiveSession(graph=graph)

# Counter Variable definition
with tf.name_scope('counter'):
    counter = tf.Variable(1, name="counter")
    tf.summary.scalar('counter', counter)

# Creation of a constant
two_op = tf.constant(2, name="const")

# Operations to perform in order to increment the variable value
new_value = tf.multiply(counter, two_op)
update = tf.assign(counter, new_value)

merged = tf.summary.merge_all()

# Initialize all variables
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    # Increment the value of the variable in a session
    sess.run(init_op)

    summary_writer = tf.summary.FileWriter("log", sess.graph)

    for i in range(5):
        summary, _ = sess.run([merged, update])
        summary_writer.add_summary(summary, i)
        print(sess.run(counter))

#Then to visualize current graph
show_graph(tf.get_default_graph().as_graph_def())

2
4
8
16
32


In [23]:
graph = tf.Graph()
with graph.as_default():
    # Instanciation of two Placeholders
    input = tf.placeholder(tf.float32)
    test = np.random.rand(10,10)
    var = tf.Variable(test)

# Graph execution, we need to feed the placeholders
with tf.Session(graph=graph) as sess:
    result = sess.run(input, feed_dict={ input: test })
    print(result.shape)
    
    print(var.get_shape()[0])
    print(var.
    
    

(10, 10)
10
Tensor("Variable/read:0", shape=(10, 10), dtype=float64)
